In [4]:
import math
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

USE_GPU = True

dtype = torch.float64 # we will be using float throughout this tutorial
device = torch.device('cuda') if (USE_GPU and torch.cuda.is_available()) else torch.device('cpu')
print('using device:', device)

using device: cuda


# The algorithm :

***Algorithm 1 :*** American Option Pricing with Multiple Neural Networks

**Result :** Functions $\Phi_{t_i}, \Psi_{t_i}$ for $i \in \{0,1, \ldots, n-1\}$

Simulate $N$ stock paths

Initialize $Y_{t_n}=X_{t_n}=f\left(S_{t_n}\right)$

for $i=n-1: 1$ do \\
&nbsp;&nbsp;&nbsp;&nbsp; Regress $\beta_{\Delta t}^{-1} Y_{t_{i+1}}$ on $S_{t_i}:$ \\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\min_{\Phi_{t_i}, \Psi_{t_i}}\left(\beta_{\Delta t}^{-1} Y_{t_{i+1}} - \Phi_{t_i}\left(S_{t_i}\right) - \Psi_{t_i}\left(S_{t_i}\right) \Delta W_{t_i}\right)^2$ \\
&nbsp;&nbsp;&nbsp;&nbsp; $Y_{t_i} = \beta_{\Delta t}^{-1} Y_{t_{i+1}} - \Psi_{t_i}\left(S_{t_i}\right) \Delta W_{t_i}$ \\
&nbsp;&nbsp;&nbsp;&nbsp; $X_{t_i} = \beta_{\Delta t}^{-1} X_{t_{i+1}} - \Psi_{t_i}\left(S_{t_i}\right) \Delta W_{t_i}$ \\
&nbsp;&nbsp;&nbsp;&nbsp; if $f\left(S_{t_i}\right) > \Phi_{t_i}\left(S_{t_i}\right)$ then \\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $Y_{t_i} = f\left(S_{t_i}\right)$ \\
&nbsp;&nbsp;&nbsp;&nbsp; end \\
&nbsp;&nbsp;&nbsp;&nbsp; if $f\left(S_{t_i}\right) > X_{t_i}$ then \\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $X_{t_i} = f\left(S_{t_i}\right)$ \\
&nbsp;&nbsp;&nbsp;&nbsp; end \\
end

Regress $\beta_{\Delta t}^{-1} Y_{t_1}$ on $S_{t_0}:$ \\
&nbsp;&nbsp;&nbsp;&nbsp; $\min \left(\beta_{\Delta t}^{-1} Y_{t_1} - \Phi_{t_0}\left(S_{t_0}\right) - \Psi_{t_0}\left(S_{t_0}\right) \Delta W_{t_0}\right)^2$ \\
&nbsp;&nbsp;&nbsp;&nbsp; $Y_{t_0} = \beta_{\Delta t}^{-1} Y_{t_1} - \Psi_{t_0}\left(S_{t_0}\right) \Delta W_{t_0}$ \\
&nbsp;&nbsp;&nbsp;&nbsp; $X_{t_0} = \beta_{\Delta t}^{-1} X_{t_1} - \Psi_{t_0}\left(S_{t_0}\right) \Delta W_{t_0}$


## American Option Pricing with Multiple Neural Networks (method 1) article [1]
Here I'll try a simple implementation of the method I of the first article :

Here we have constant interest rate so the discount factor is $\exp(-rT)$, and the stock dynamics are modelled with Geometric Brownian Motion (GBM).

$\large dS_t = rS_tdt+\sigma S_tdW_t$

Let's simulate this GBM process by simulating variables of the natural logarithm process of the stock price $x_t = ln(S_t)$, which is normally distributed. For the dynamics of the natural logarithm process of stock prices under GBM model you need to use Ito's calculus.
$\large dx_t = \nu dt+\sigma dW_t ,  \nu = r - \frac{1}{2} \sigma ^ 2$

We can then discretize the Stochastic Differential Equation (SDE) by changing the infinitesimals $dx, dt, dz$ into small steps $\Delta x, \Delta t, \Delta W$.

$\large \Delta x = \nu  \Delta t+\sigma \Delta W$

This is the exact solution to the SDE and involves no approximation.

$\large x_{t+\Delta t} = x_{t} + \nu (\Delta t)+\sigma (W_{t+\Delta t}- W_t)$

In terms of the stock price S, we have:

$\large S_{t+\Delta t} = S_{t} \exp( \nu \Delta t + \sigma (W_{t+\Delta t}- W_t) )$

Where $(W_{t+\Delta t}- W_t) \sim N(0,\Delta t) \sim \sqrt{\Delta t} N(0,1) \sim \sqrt{\Delta t} \epsilon_i$


\\


In [28]:
def pay_off(S,r,K,dt):
  return torch.maximum(K- S,torch.zeros_like(S))

def simulate_paths(M, T, n, r, vol, S, K, dt):
    nudt = (r - 0.5 * vol**2) * dt
    lnS = np.log(S)
    # Méthode de Monte Carlo
    Z = np.random.normal(size=(M,n))
    dW=np.sqrt(dt) * Z  #it's the simulation of the dWt_i we'll need in each iteration
    delta_lnSt = nudt + vol*dW

    LnS_s = np.zeros([M, n + 1])

    # Set the initial values
    LnS_s[:, 0] = lnS
    # Compute cumulative sums using cumsum
    LnS_s[:, 1:] = np.cumsum(delta_lnSt, axis=1)
    LnS_s[:,1:] +=lnS


    S = np.exp(LnS_s)
    S_tensor = torch.tensor(S, device = device ,dtype=dtype)
    dW_tensor = torch.tensor(dW, device = device ,dtype=dtype)
    return S_tensor, dW_tensor

#Parametres
T = 1
n=50
dt = T/n  #les t_i seront donc les i*dt.


S = 36          # Prix de l'action
K = 40           # Prix d'exercice
vol = 0.2       # Volatilité (%)
r = 0.06            # Taux sans risque (%)
M = 100000 # Nombre de simulations


S,dW=simulate_paths(M, T, n, r, vol, S, K, dt)
X=torch.zeros([M,n+1], device = device ,dtype=dtype)
Y=torch.zeros_like(X, device = device ,dtype=dtype)


X[ :, n]=pay_off(S[:,n],r,K,dt)
Y[ :, n]=X[ :, n]

beta_dt=math.exp(-r*dt)
epsilon = 1e-8 #for all division tasks


In [33]:
class SimpleModel1(nn.Module):
    def __init__(self):
        super(SimpleModel1, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 1)
        )

    def forward(self, x):
        return self.model(x)

class SimpleModel2(nn.Module):
    def __init__(self):
        super(SimpleModel2, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(1, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 1)
        )

    def forward(self, x):
        return self.model(x)

class ModelManager():
    def __init__(self, model):
        self.model = model
        self.loss_fn = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)

    def train(self, i , inputs, outputs, epochs=5, batch_size=32, validation_split=0.2):
        torch.cuda.empty_cache()
        dataset = TensorDataset(inputs, outputs)
        train_size = int((1 - validation_split) * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

        self.model.train()
        for epoch in range(epochs):
            total_loss = 0
            for batch_inputs, batch_labels in train_dataloader:
                self.optimizer.zero_grad()
                outputs = self.model(batch_inputs)
                loss = self.loss_fn(outputs, batch_labels)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()


            val_loss = 0
            if epoch % 3 == 0:
                self.model.eval()
                with torch.no_grad():
                    for S_batch, labels_batch in val_dataloader:
                          outputs = self.model(S_batch)
                          val_loss += self.loss_fn(labels_batch, outputs).item()

                print(f"for the {i} th iteration and epoch {epoch+1}/{epochs}, Training Loss: {total_loss/len(train_dataset)}, Validation Loss: {val_loss/val_size}")


    def predict(self, inputs):
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(inputs)
        return outputs

model1 = SimpleModel1().to(device = device ,dtype=dtype)
model2 = SimpleModel2().to(device = device ,dtype=dtype)

manager1 = ModelManager(model1)
manager2 = ModelManager(model2)





In [34]:
for i in range(n-1,0,-1):
    #determination of phi and psi in two steps

    # model 1
    concatenated_inputs= torch.concatenate([S[:,i].reshape(-1,1),dW[:,i].reshape(-1,1)],axis=1)
    manager1.train(i ,concatenated_inputs,beta_dt*Y[:,i+1], epochs=10, batch_size=15)
    general_fct_S_W = manager1.predict(concatenated_inputs) # Y ~ f(S,W) #as a function of S, W

    #transformation on the outputs
    concatenated_inputs_0 = torch.concatenate([S[:,i].reshape(-1,1),torch.zeros_like(dW[:,i].reshape(-1,1))],axis=1) #or just torch.zeros(M,1)
    phi_S = manager1.predict(concatenated_inputs_0) # predit f(S, 0) = phi(S)
    psi_S_prime= (general_fct_S_W-phi_S)/(dW[:,i]+epsilon).reshape(-1,1) # (f(S,W)-phi(S))/W

    # model 2
    manager2.train(i , S[:,i].reshape(-1,1),psi_S_prime, epochs=10, batch_size=15) # (f(S,W)-phi(S))/W ~ g(S) as a function of g(S), we are litteraly computing the df/dW(S,0) !
    psi_S= manager2.predict(S[:,i].reshape(-1,1)) #and now g(S) = psi(S)

    X[ :, i]=beta_dt*X[:,i+1]-psi_S.reshape(M) *dW[:,i]
    Y[ :, i]=beta_dt*Y[:,i+1]-psi_S.reshape(M) *dW[:,i]
    Z=pay_off(S[:,i],r,K,dt)


    Y[ :, i] = torch.where(Z> phi_S.reshape(M), Z, Y[ :, i])
    X[ :, i] = torch.where(Z> X[:,i], Z, X[ :, i])


#Pour i =0 :
#determination of phi and psi in two steps
i =0
# model 1
concatenated_inputs= torch.concatenate([S[:,i].reshape(-1,1),dW[:,i].reshape(-1,1)],axis=1)
manager1.train(i , concatenated_inputs,beta_dt*Y[:,i+1], epochs=10, batch_size=15)
general_fct_S_W = manager1.predict(concatenated_inputs)

#transformation on the outputs
concatenated_inputs_0= torch.concatenate([S[:,i].reshape(-1,1),torch.zeros_like(dW[:,i].reshape(-1,1))],axis=1)
phi_S = manager1.predict(concatenated_inputs_0)
psi_S_prime= (general_fct_S_W-phi_S)/(dW[:,i]+epsilon).reshape(-1,1)

# model 2
manager2.train(i, S[:,i].reshape(-1,1),psi_S_prime, epochs=10, batch_size=15)
psi_S= manager2.predict(S[:,i].reshape(-1,1))

X[ :, i]=beta_dt*X[:,i+1]-psi_S.reshape(M) *dW[:,i]
Y[ :, i]=beta_dt*Y[:,i+1]-psi_S.reshape(M) *dW[:,i]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([15])) that is different to the input size (torch.Size([15, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([15, 1])) that is different to the input size (torch.Size([15])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, t

for the 49 th iteration and epoch 1/10, Training Loss: 2.50501713244713, Validation Loss: 2.125764443026513
for the 49 th iteration and epoch 4/10, Training Loss: 1.4170994822364824, Validation Loss: 1.9431932053762941
for the 49 th iteration and epoch 7/10, Training Loss: 1.3352874569230448, Validation Loss: 2.020372720593801
for the 49 th iteration and epoch 10/10, Training Loss: 1.4834127192226305, Validation Loss: 1.9416574384914154
for the 49 th iteration and epoch 1/10, Training Loss: 0.06875064680406713, Validation Loss: 0.05499187221114125
for the 49 th iteration and epoch 4/10, Training Loss: 0.0018758473840112429, Validation Loss: 0.00613116815156593
for the 49 th iteration and epoch 7/10, Training Loss: 0.00031747501065201595, Validation Loss: 0.000884827270101568
for the 49 th iteration and epoch 10/10, Training Loss: 7.365965221000593e-05, Validation Loss: 7.804114813050447e-05
for the 48 th iteration and epoch 1/10, Training Loss: 1.377566847010927, Validation Loss: 1.813

In [35]:
#Monté Carlo
u0=torch.mean(X[:,0]).cpu().numpy()
l0=torch.mean(Y[:,0]).cpu().numpy()
print(u0)
print(l0)


7.248307879543311
3.990814446003303


In [36]:
print(np.abs(u0-l0))
print(4.4887-4.4762)
print(np.abs(np.abs(u0-l0)-(4.4887-4.4762)))

3.2574934335400076
0.01249999999999929
3.2449934335400084
